Voy a tratar de aplicar la deteccion de outliers por lejania o distancia a los datos de este set sobre duraznos.

*Librerias*

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
import Ourfunctions as ofn
import scikitplot as skplt 
from sklearn.model_selection import train_test_split
np.random.seed(123)

**Ahora leemos el csv, luego de haber corrido el from_exceltoCsv.py**

In [ ]:
datos = pd.read_csv("duraznosfinal.csv")
#Eliminamos la primer columna que no nos aporta nada.
datos.drop(datos.columns[0],axis = 1, inplace = True)
datos.shape

In [ ]:
sns.set_style("whitegrid")
flierprops = dict(marker='o', markerfacecolor='red', markersize=5,  markeredgecolor='None')
sns.boxplot(data= datos,y = "Peso",x="Zona",flierprops = flierprops)

In [ ]:
#split into independent variables and target variable\n",
x = datos.drop(['Zona','Finca','Diámetro Sutural','Diámetro Contrasutural'], axis=1)
#outliers son 9
n_outliers = 9
ground_truth = np.ones(len(x), dtype=int)
ground_truth[-n_outliers:] = -1
# show the shape of x and y  
print("x shape: {}".format(x.shape))

In [ ]:
clf  = LocalOutlierFactor(n_neighbors = 20,contamination = 0.1) #20 neighbors es una recomendación que dan todos los libros como numero "magico"

In [ ]:
x_train ,x_test = train_test_split(x,test_size=0.5)       #test_size=0.5(whole_data)
ofn.ClasificarOutliersVect(x)

In [ ]:
y_pred = clf.fit_predict(x)
n_errors = (y_pred != ground_truth).sum()
X_scores = clf.negative_outlier_factor_